In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import re
import sys
import json
from IPython.display import Image, display
os.chdir('..')
import pickle
import numpy as np

In [ ]:
with open('config/data-params.json') as fh:
    data_cfg = json.load(fh)

# Training Data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
dim = 'pol_rightness'

In [ ]:
df = pd.read_csv(os.path.join(data_cfg['training_data_path'], 'data.csv')).drop(columns=['Unnamed: 0'])
def remove_hashtags_and_ats(x):
    return x.replace('#', '').replace('@', '')
df['text'] = df['text'].apply(remove_hashtags_and_ats)
df.head()

In [ ]:
df.shape[0]

In [ ]:
df[dim].value_counts()

# Classifier

#### Create classifier

In [ ]:
MIN_DF = 0.000002
MIN_DF*df.shape[0]

In [ ]:
count_vect = CountVectorizer(stop_words='english')
tfidf_transformer = TfidfTransformer()
clf = MultinomialNB(fit_prior=False)

Cross validation score

In [ ]:
X, y = df['text'], df[dim]
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(X)

In [ ]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    text_clf = Pipeline([('vect', count_vect), ('tfidf', tfidf_transformer), ('clf', clf)])
    text_clf.fit(X_train, y_train)
    y_pred = text_clf.predict(X_test)
    class_acc = np.mean(y_pred == y_test)
    reg_error = np.mean(abs(y_pred - y_test))
    print('Class_acc: {}'.format(class_acc))
    print('Regression error: {}'.format(reg_error))

### Check Regression Error

In [ ]:
y_pred = text_clf.predict(X_test)

In [ ]:
avg_error = np.mean(abs(y_pred - y_test))
avg_error

# User Test Data

Train Model

In [ ]:
# Reading in user tweets
tweets = {}
for tweet_id in data_cfg['tweet_ids']:
    path = os.path.join(data_cfg['output_user_data_path'], 'tweet_{}.csv'.format(tweet_id))
    tweet = pickle.load(open(path, 'rb'))
    tweets[tweet_id] = tweet
    for key, value in tweets.items():
        for user_id in list(value['user_ids'].keys()):
            value['user_ids'][user_id] = pd.read_csv(os.path.join(data_cfg['output_user_data_path'], 'user_{}_tweets.csv'.format(user_id)))

In [ ]:
# # Going through all the users of all the tweets
# for tweet_id in tweets.keys():
#     user_ids = list(tweets[tweet_id]['user_ids'].keys())
#     for user_id in user_ids:
#         df = pd.read_csv(os.path.join(data_cfg['output_user_data_path'], 'user_{}_tweets.csv'.format(user_id)))
#         def remove_hashtags_and_ats(x):
#             return x.replace('#', '').replace('@', '')
#         df['text'] = df['text'].apply(remove_hashtags_and_ats)
#         if df.shape[0] != 0:
#             print('user_id: {}'.format(user_id))
#             print(text_clf.predict_proba(df['text']))

In [ ]:
for tweet_id in data_cfg['tweet_ids']:
    user_ids = list(tweets[tweet_id]['user_ids'].keys())
    print('Tweet ID: {}'.format(tweet_id))
    print(user_ids)
    print()
    print()

In [ ]:
user_id = '1061497789'
df = pd.read_csv(os.path.join(data_cfg['output_user_data_path'], 'user_{}_tweets.csv'.format(user_id)))
def remove_hashtags_and_ats(x):
    return x.replace('#', '').replace('@', '')
df['text'] = df['text'].apply(remove_hashtags_and_ats)
display(df['text'])
if df.shape[0] != 0:
    print('user_id: {}'.format(user_id))
    print(text_clf.predict(df['text']))